<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/Analitics/part_01/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata

In [3]:
# Подключаемся к базе postgres на https://supabase.com/
engine = create_engine(userdata.get('supabase'), client_encoding='utf8')

In [2]:
# пишем функцию-обертку для удобного получения данных в виде pandas df
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

## Нужно собрать базу контактов для email-рассылки.
Почта указана абсолютно у всех, а вот имя/
фамилия first_name и last_name - не у всех. В результате нужно
получить 2 поля: email, full_name. Здесь email - почтовый адрес
из базы, а full_name - это имя и фамилия, написанные через
пробел. Причем если имя в базе не указано, то вместо
связки имя фамилия нужно вывести Дорогой друг.


In [29]:

sql = """
SELECT
  email,
  (CASE WHEN
  coalesce(first_name, '') || ' ' || coalesce(last_name, '') = ' ' then 'Дорогой друг'
  else coalesce(first_name, '') || ' ' || coalesce(last_name, '') end) AS full_name
FROM simulative.user
"""
data = select(sql)

In [30]:
data

,email,full_name
0,jbsnjbf@gmail.com,Дорогой друг
1,fjbis@yandex.ru,Элина Полякова
2,jjxbfox@bk.ru,Эмилия Афанасьев
3,isfsobm.nseo@gmail.com,Дорогой друг
4,koonbnanojj.jofbx@yandex.ru,Эмилия Фролова
...,...,...
2769,bb.jk@bk.ru,Илья Иван
2770,nbjsbjds@mail.ru,Яна Сухарева
2771,cookboo@gmail.com,Артём Хохлова
2772,fdsnbfo@example.org,Дорогой друг


К сожалению, мы можем рассылать письма не на все почтовые
серверы. Нужно произвести валидацию почтовых адресов:
адрес должен быть либо в российской доменной зоне `.ru`, либо
на домене `gmail.com`. Остальные ящики, к сожалению, не
сработают. В результате нужно вывести отфильтрованный
столбец `email`.


In [31]:
sql = """
SELECT
  email,
  (CASE WHEN
  coalesce(first_name, '') || ' ' || coalesce(last_name, '') = ' ' then 'Дорогой друг'
  else coalesce(first_name, '') || ' ' || coalesce(last_name, '') end) AS full_name
FROM simulative.user
WHERE email LIKE '%@%.ru' OR email LIKE '%@gmail.com'
"""
data = select(sql)

In [32]:
data

,email,full_name
0,jbsnjbf@gmail.com,Дорогой друг
1,fjbis@yandex.ru,Элина Полякова
2,jjxbfox@bk.ru,Эмилия Афанасьев
3,isfsobm.nseo@gmail.com,Дорогой друг
4,koonbnanojj.jofbx@yandex.ru,Эмилия Фролова
...,...,...
2553,dsbobenob@mail.ru,Даниил Гончарова
2554,abd-mbmb@ya.ru,Константин Хохлова
2555,bb.jk@bk.ru,Илья Иван
2556,nbjsbjds@mail.ru,Яна Сухарева


На платформе у нас есть внутренняя классификация
пользователей. Она зависит от значения `score` - баллы, которые
пользователь получает за решение задач. Нужно написать
запрос, в котором будут столбцы `email`, `score`, `class`.

Поле `class` должно вычисляться так:
* если у пользователя` 0 <= score < 20 - группа D;
* если 20 <= score < 100 - группа C;
* если 100 <= score < 500 - группа B;
* если больше 500 - группа A.

Нам это важно знать, чтобы предложить пользователю нужную
активность.